In [1]:
import sys
import os
sys.path.append(os.path.abspath(".."))
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

In [3]:
data = [
    [101, 'John'],
    [101, 'James']
]

columns = ['SID','SName']

df_data = pd.DataFrame(data, columns=columns)
print(df_data)

   SID  SName
0  101   John
1  101  James


In [4]:
from src.database.db_connection import *

In [5]:
engine = create_engine_for_db()

In [6]:
fd_constraint1 = [(['SID'],['SName'])]

In [7]:
from src.utilities.step1_simplify_fds import *

In [8]:
fd_constraint1 = simplify_FDs(fd_constraint1)

In [9]:
fd_constraint1

[(['SID'], ['SName'])]

In [10]:
from src.utilities.step2_clustering_and_generating_repairs import *

In [11]:
df_data = preprocess_data(df_data)

In [12]:
df_data

,SID,SName,uuid
0,101,John,1
1,101,James,2


In [13]:
rv_definitions,rv_probabilities,consistent_rows = create_rv_definitions(df_data,fd_constraint1)

In [14]:
rv_definitions

{'fd1SName1=1': ('James', {2}),
 'fd1SName1=2': ('James', {1, 2}),
 'fd1SName1=3': ('John', {1}),
 'fd1SName1=4': ('John', {1, 2})}

In [15]:
rv_probabilities

{'fd1SName1=1': 0.25,
 'fd1SName1=2': 0.25,
 'fd1SName1=3': 0.25,
 'fd1SName1=4': 0.25}

In [16]:
from src.utilities.step3_convert_back_to_db import *

In [17]:
repaired_rows = create_repaired_rows(rv_definitions,df_data,consistent_rows)

In [18]:
columns = [col for col in df_data.columns if col != 'uuid'] + ['_sentences']
repaired_df = pd.DataFrame(repaired_rows, columns=columns)

In [19]:
repaired_df

,SID,SName,_sentences
0,101,James,fd1SName1=1 | fd1SName1=2
1,101,James,fd1SName1=2
2,101,John,fd1SName1=3 | fd1SName1=4
3,101,John,fd1SName1=4


In [20]:
from src.utilities.step4_group_similar_sentences import *

In [21]:
repaired_df = group_similar_rows_together(repaired_df)

In [22]:
repaired_df

,SID,SName,_sentences
0,101,James,(fd1SName1=1 | fd1SName1=2) | (fd1SName1=2)
1,101,John,(fd1SName1=3 | fd1SName1=4) | (fd1SName1=4)
